In [18]:
%pip install pretty_midi
%pip install tensorflow
%pip install music21


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [19]:
%pip uninstall fluidsynth -y
%pip install --upgrade fluidsynth 

Found existing installation: fluidsynth 0.2
Uninstalling fluidsynth-0.2:
  Successfully uninstalled fluidsynth-0.2
Note: you may need to restart the kernel to use updated packages.
  Using cached fluidsynth-0.2-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [20]:
import pandas as pd
import numpy as np
import pretty_midi
import pathlib
import collections
import datetime
import glob
import music21
import tensorflow as tf
import fluidsynth


In [56]:
import my_functions
import importlib

In [22]:
importlib.reload(my_functions)

<module 'my_functions' from '/Users/teovargo/MusicGenProject/my_functions.py'>

In [23]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# Sampling rate for audio playback
_SAMPLING_RATE = 16000

In [24]:
data_dir=pathlib.Path('Jazzomat_midi_folder')
filenames = glob.glob(str(data_dir/"*.mid"))
print("Number of Files:", len(filenames))
sorted_filenames = sorted(filenames)
#print(sorted_filenames[:5])
sample_file = sorted_filenames[0]
print(sample_file)


Number of Files: 456
Jazzomat_midi_folder/ArtPepper_Anthropology_FINAL.mid


In [25]:
pm = pretty_midi.PrettyMIDI(sample_file)


In [26]:
def midi_to_notes(midi_file: str):
  midi_data = pretty_midi.PrettyMIDI(midi_file)
  instrument = midi_data.instruments[0]
  notes = collections.defaultdict(list)

  sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
  prev_start = sorted_notes[0].start

  for note in sorted_notes:
    start = note.start
    end = note.end
    notes['pitch'].append(note.pitch)
    notes['note_name'].append(pretty_midi.note_number_to_name(note.pitch))
    notes['start'].append(start)
    notes['end'].append(end)
    notes['step'].append(start-prev_start)
    notes['duration'].append(end - start)
    prev_start = start

  return pd.DataFrame({name: np.array(value) for name, value in notes.items()})

In [347]:
#Create Training Dataset by concatenating multiple songs worth of notes
#num_files = 10
all_notes = []
for f in sorted_filenames[0:5]:
  notes = midi_to_notes(f)
  all_notes.append(notes)

all_notes = pd.concat(all_notes)

n_notes = len(all_notes)
print("Number of Parsed Notes:", n_notes)

Number of Parsed Notes: 2725


In [27]:
# This loads up all the notes in the dataset
pitchInst = my_functions.extract_notes()

# Calc the gap between note end and next not start
pitchInst['step'] = pitchInst['end'] - pitchInst['start'].shift(1)

# Fix up the first one cause of the wacky shift problem
pitchInst['step'][0] = 0

# Calculate the inverval between successive notes
pitchInst['interval'] = (pitchInst['pitch'] - pitchInst['pitch'].shift(1))

# Fix up the first one cause of the wacky shift problem
pitchInst['interval'][0] = 0

# apply a contour function
pitchInst['contour'] = pitchInst['interval'].apply(my_functions.contour)

n_notes = len(pitchInst)
pitchInst.head()

/var/folders/0n/vghykfw130n6133zv_3yq7_c0000gn/T/ipykernel_89334/3041708538.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pitchInst['step'][0] = 0
/var/folders/0n/vghykfw130n6133zv_3yq7_c0000gn/T/ipykernel_89334/3041708538.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pitchInst['interval'][0] = 0


,melid,pitch,start,end,duration,instrument,step,interval,contour
0,1,65.0,10.343492,10.482268,0.138776,cl,0.000000,0.0,0.0
1,1,63.0,10.637642,10.808889,0.171247,cl,0.465397,-2.0,-2.0
2,1,58.0,10.843719,10.924989,0.081270,cl,0.287347,-5.0,-10.0
3,1,61.0,10.948209,11.183311,0.235102,cl,0.339592,3.0,3.0
4,1,63.0,11.232653,11.363265,0.130612,cl,0.415057,2.0,2.0


In [ ]:
train_sub = pitchInst[pitchInst['melid'] > 255]
#train_sub['step', 'interval', 'contour'][] = 0
key_order = ['pitch', 'contour', 'step', 'duration']
train_notes = np.stack([train_sub[key] for key in key_order], axis=1)


array([[ 6.90000000e+01,  2.00000000e+01, -4.48958277e+01,
         2.59183673e-01],
       [ 7.00000000e+01,  1.00000000e+00,  3.12244898e-01,
         5.10204082e-02],
       [ 7.20000000e+01,  2.00000000e+00,  2.34693878e-01,
         1.73469388e-01],
       [ 7.20000000e+01,  0.00000000e+00,  3.71428571e-01,
         1.61224490e-01],
       [ 7.20000000e+01,  0.00000000e+00,  6.32653061e-01,
         3.08163265e-01]])

In [39]:
notes_ds = tf.data.Dataset.from_tensor_slices(train_notes)
notes_ds.element_spec

TensorSpec(shape=(4,), dtype=tf.float64, name=None)

In [ ]:
# from tensorFlow MusGen tutorial 
def create_sequences(
    dataset: tf.data.Dataset,
    seq_length: int,
    vocab_size: int,
) -> tf.data.Dataset:
  """Returns TF Dataset of sequence and label examples."""
  seq_length = seq_length+1

  # Take 1 extra for the labels
  windows = dataset.window(seq_length, shift=1, stride=1,
                              drop_remainder=True)

  # `flat_map` flattens the" dataset of datasets" into a dataset of tensors
  flatten = lambda x: x.batch(seq_length, drop_remainder=True)
  sequences = windows.flat_map(flatten)

  # Normalize note pitch (is this a good idea?)
  def scale_pitch(x):
    x = x/[vocab_size,1.0,1.0,1.0]
    return x

  # Split the labels
  def split_labels(sequences):
    inputs = sequences[:-1]
    labels_dense = sequences[-1]
    labels = {key:labels_dense[i] for i,key in enumerate(key_order)}

    return scale_pitch(inputs), labels
    #return inputs, labels

  return sequences.map(split_labels, num_parallel_calls=tf.data.AUTOTUNE)

In [41]:
seq_length = 10
vocab_size = 128
seq_ds = create_sequences(notes_ds, seq_length, vocab_size)
seq_ds.element_spec

(TensorSpec(shape=(10, 4), dtype=tf.float64, name=None),
 {'pitch': TensorSpec(shape=(), dtype=tf.float64, name=None),
  'contour': TensorSpec(shape=(), dtype=tf.float64, name=None),
  'step': TensorSpec(shape=(), dtype=tf.float64, name=None),
  'duration': TensorSpec(shape=(), dtype=tf.float64, name=None)})

In [42]:
for seq, target in seq_ds.take(1):
  print('sequence shape:', seq.shape)
  print('sequence elements (first 10):', seq[0: 10])
  print()
  print('target:', target)

sequence shape: (10, 4)
sequence elements (first 10): tf.Tensor(
[[  0.5390625   20.         -44.89582766   0.25918367]
 [  0.546875     1.           0.3122449    0.05102041]
 [  0.5625       2.           0.23469388   0.17346939]
 [  0.5625       0.           0.37142857   0.16122449]
 [  0.5625       0.           0.63265306   0.30816327]
 [  0.5625       0.           0.50816327   0.17959184]
 [  0.5078125  -10.           2.2755102    0.22244898]
 [  0.5625      10.           0.46326531   0.2244898 ]
 [  0.5625       0.           0.39591837   0.13877551]
 [  0.5625       0.           0.53265306   0.19795918]], shape=(10, 4), dtype=float64)

target: {'pitch': <tf.Tensor: shape=(), dtype=float64, numpy=72.0>, 'contour': <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, 'step': <tf.Tensor: shape=(), dtype=float64, numpy=0.7918367346938773>, 'duration': <tf.Tensor: shape=(), dtype=float64, numpy=0.5714285714285714>}


2024-11-22 11:15:51.036640: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [43]:
batch_size = 64
buffer_size = n_notes - seq_length  # the number of items in the dataset
train_ds = (seq_ds
            .shuffle(buffer_size)
            .batch(batch_size, drop_remainder=True)
            .cache()
            .prefetch(tf.data.experimental.AUTOTUNE))

In [44]:
train_ds.element_spec

(TensorSpec(shape=(64, 10, 4), dtype=tf.float64, name=None),
 {'pitch': TensorSpec(shape=(64,), dtype=tf.float64, name=None),
  'contour': TensorSpec(shape=(64,), dtype=tf.float64, name=None),
  'step': TensorSpec(shape=(64,), dtype=tf.float64, name=None),
  'duration': TensorSpec(shape=(64,), dtype=tf.float64, name=None)})

In [45]:
def mse_with_positive_pressure(y_true: tf.Tensor, y_pred: tf.Tensor):
  mse = (y_true - y_pred) ** 2
  positive_pressure = 10 * tf.maximum(-y_pred, 0.0)
  return tf.reduce_mean(mse + positive_pressure)

In [49]:
input_shape = (seq_length, 4)
learning_rate = 0.05

#change input layers here? to suit contour
inputs = tf.keras.Input(input_shape)
x = tf.keras.layers.LSTM(128)(inputs)
#and here?
outputs = {
  'pitch': tf.keras.layers.Dense(128, name='pitch')(x),
  'step': tf.keras.layers.Dense(1, name='step')(x),
  'duration': tf.keras.layers.Dense(1, name='duration')(x),
}

model = tf.keras.Model(inputs, outputs)

loss = {
      'pitch':tf.keras.losses.SparseCategoricalCrossentropy(
          from_logits=True),
      'step': mse_with_positive_pressure,
      'duration': mse_with_positive_pressure,
}

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(loss=loss, optimizer=optimizer)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 10, 4)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 128)       │     68,096 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ duration (Dense)    │ (None, 1)         │        129 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pitch (Dense)       │ (None, 128)       │     16,512 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ step (Dense)        │ (None, 1)         │        129 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 84,866 (331.51 KB)

 Trainable params: 84,866 (331.51 KB)

 Non-trainable params: 0 (0.00 B)

In [50]:
model.compile(
    loss=loss,
    loss_weights={
        'pitch': 0.10,
        'step': 1.0,
        'duration':1.0,
    },
    optimizer=optimizer,
)

In [51]:
model.evaluate(train_ds, return_dict=True)


1223/1223 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - duration_loss: 0.1081 - loss: 34.3720 - pitch_loss: 0.4843 - step_loss: 33.7796


2024-11-22 11:37:10.228993: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/teovargo/anaconda3/lib/python3.11/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


{'duration_loss': 0.10964304953813553,
 'loss': 28.565135955810547,
 'pitch_loss': 0.48380714654922485,
 'step_loss': 27.971683502197266}

In [52]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='./training_checkpoints/ckpt_{epoch}.weights.h5',
        save_weights_only=True),
    tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=5,
        verbose=1,
        restore_best_weights=True),
]

In [53]:
%%time
epochs = 10

history = model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
)

Epoch 1/10
1223/1223 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - duration_loss: 0.2277 - loss: 34.4675 - pitch_loss: 0.3559 - step_loss: 33.8839
Epoch 2/10
1223/1223 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - duration_loss: 0.0423 - loss: 33.6434 - pitch_loss: 0.3392 - step_loss: 33.2620
Epoch 3/10
  27/1223 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - duration_loss: 0.0294 - loss: 99.2430 - pitch_loss: 0.3334 - step_loss: 98.8802  

2024-11-22 11:37:35.703909: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


1223/1223 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - duration_loss: 0.0458 - loss: 33.6366 - pitch_loss: 0.3368 - step_loss: 33.2540
Epoch 4/10
1223/1223 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - duration_loss: 0.0484 - loss: 33.6374 - pitch_loss: 0.3357 - step_loss: 33.2534
Epoch 5/10
1223/1223 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - duration_loss: 0.0482 - loss: 33.6380 - pitch_loss: 0.3362 - step_loss: 33.2536
Epoch 6/10
1223/1223 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - duration_loss: 0.0608 - loss: 33.6765 - pitch_loss: 0.3372 - step_loss: 33.2785
Epoch 7/10
   9/1223 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - duration_loss: 0.0253 - loss: 175.0772 - pitch_loss: 0.3129 - step_loss: 174.7390

2024-11-22 11:38:06.556807: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


1223/1223 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - duration_loss: 0.0544 - loss: 33.6511 - pitch_loss: 0.3357 - step_loss: 33.2609
Epoch 8/10
1223/1223 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - duration_loss: 0.0470 - loss: 33.6162 - pitch_loss: 0.3059 - step_loss: 33.2634
Epoch 9/10
1223/1223 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - duration_loss: 0.0654 - loss: 33.6196 - pitch_loss: 0.2915 - step_loss: 33.2626
Epoch 10/10
1223/1223 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - duration_loss: 0.0549 - loss: 33.6073 - pitch_loss: 0.2883 - step_loss: 33.2640
Restoring model weights from the end of the best epoch: 10.
CPU times: user 1min 29s, sys: 20.6 s, total: 1min 49s
Wall time: 1min 18s


In [ ]:
def predict_next_note(
    notes: np.ndarray,
    model: tf.keras.Model,
    temperature: float = 1.0) -> tuple[int, float, float]:
  """Generates a note as a tuple of (pitch, step, duration), using a trained sequence model."""

  assert temperature > 0

  # Add batch dimension
  inputs = tf.expand_dims(notes, 0)

  predictions = model.predict(inputs)
  pitch_logits = predictions['pitch']
  #contour = predictions['contour']
  step = predictions['step']
  duration = predictions['duration']

  pitch_logits /= temperature 
  pitch = tf.random.categorical(pitch_logits, num_samples=1) 
  pitch = tf.squeeze(pitch, axis=-1) 
  #contour = tf.squeeze(contour, axis=-1) 
  duration = tf.squeeze(duration, axis=-1)
  step = tf.squeeze(step, axis=-1)

  # `step` and `duration` values should be non-negative
  #pitch = tf.maximum(0,128)
  step = tf.maximum(0, step)
  duration = tf.maximum(0, duration)
  #contour = tf.maximum(-20, 20)

  return int(pitch), float(step), float(duration)

In [57]:
raw_notes = midi_to_notes(sample_file)
#raw_notes.head()
raw_notes['interval'] = (raw_notes['pitch'] - raw_notes['pitch'].shift(1))
raw_notes['interval'][0] = 0
raw_notes['contour'] = raw_notes['interval'].apply(my_functions.contour)
raw_notes.head()

/var/folders/0n/vghykfw130n6133zv_3yq7_c0000gn/T/ipykernel_89334/1643645790.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_notes['interval'][0] = 0


,pitch,note_name,start,end,step,duration,interval,contour
0,65,F4,10.342443,10.480856,0.000000,0.138413,0.0,0.0
1,63,D#4,10.634965,10.806198,0.292522,0.171233,-2.0,-2.0
2,58,A#3,10.840444,10.920353,0.205479,0.079908,-5.0,-10.0
3,61,C#4,10.943184,11.177201,0.102739,0.234018,3.0,3.0
4,63,D#4,11.225717,11.355569,0.282534,0.129851,2.0,2.0


In [84]:

temperature = 2.0
num_predictions = 120

sample_notes = np.stack([raw_notes[key] for key in key_order], axis=1)

# The initial sequence of notes; pitch is normalized similar to training sequences
input_notes = (
    sample_notes[:seq_length] / np.array([vocab_size, 1, 1, 1]))

generated_notes = []
prev_start = 0
#contour = 0  #initial contour should be last contour of input sequence
for i in range(num_predictions): #THERE ARE PROBLEMS HERE
  pitch, step, duration = predict_next_note(input_notes, model, temperature)
  start = prev_start + step
  end = start + duration
  interval = pitch - input_note[0]
  contour = my_functions.contour(interval)
  input_note = (pitch, contour, step, duration)
  generated_notes.append((*input_note, start, end))
  input_notes = np.delete(input_notes, 0, axis=0)
  input_notes = np.append(input_notes , np.expand_dims(input_note, 0), axis=0) 
  prev_start = start
  

generated_notes = pd.DataFrame(
    generated_notes, columns=(*key_order, 'start', 'end'))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
input_notes.choose

In [82]:
input_notes = (
    sample_notes[:seq_length] / np.array([vocab_size, 1, 1, 1]))
new_note = predict_next_note(input_notes, model, temperature=2)
print(sample_notes)
print(new_note)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
[[ 65.           0.           0.           0.13841294]
 [ 63.          -2.           0.29252219   0.1712325 ]
 [ 58.         -10.           0.205479     0.0799085 ]
 ...
 [ 63.          -2.           0.114155     0.10416644]
 [ 62.          -1.           0.1255705    0.10987419]
 [ 70.          10.           0.16267088   0.18692881]]
(69, 0.3112975060939789, 0.1401246041059494)


In [83]:
generated_notes.head()

,pitch,contour,step,duration,start,end
0,62,-20,0.311298,0.140125,0.311298,0.451422
1,78,20,0.606952,0.053876,0.918249,0.972125
2,81,3,0.310890,0.169608,1.229140,1.398748
3,80,-1,0.310890,0.169608,1.540030,1.709638
4,74,-10,0.310890,0.169608,1.850921,2.020529


In [ ]:
instrument =pretty_midi.instruments[0]
instrument_name = pretty_midi.program_to_instrument_name(instrument.program)
example_file = 'example.midi'
example_pm = my_functions.notes_to_midi(generated_notes, out_file=example_file, instrument_name=instrument_name)

NameError: name 'pm' is not defined

In [15]:
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=30):
  waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
  # Take a sample of the generated waveform to mitigate kernel resets
  waveform_short = waveform[:seconds*_SAMPLING_RATE]
  return display.Audio(waveform_short, rate=_SAMPLING_RATE)

In [16]:
_SAMPLING_RATE=44100
display_audio(pm)

ImportError: fluidsynth() was called but pyfluidsynth is not installed.